In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing libraries

In [2]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

     |████████████████████████████████| 358kB 17.3MB/s 
     |████████████████████████████████| 102kB 12.0MB/s 
     |████████████████████████████████| 2.1MB 29.1MB/s 
     |████████████████████████████████| 20.1MB 1.5MB/s 
     |████████████████████████████████| 225kB 53.7MB/s 
     |████████████████████████████████| 337kB 55.9MB/s 
     |████████████████████████████████| 778kB 47.0MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 3.3MB 26.7MB/s 
     |████████████████████████████████| 901kB 49.4MB/s 
     |████████████████████████████████| 245kB 60.3MB/s 
     |████████████████████████████████| 112kB 61.6MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 798kB 45.0MB/s 
     |████████████████████████████████| 1.2MB 44.9MB/s 
     |████████████████████████████████| 983kB 44.0MB/s 
     |████████████████████████████████| 19.7MB 164kB/s 
  Created wheel for word2number: filename=word2num

     |████████████████████████████████| 23.9MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149825 sha256=21e414edcb55d12f0a350a776df9f2ba0fae7bccc45cf2cc5ec065215df3a466
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [12]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

In [3]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [ ]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [4]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [5]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [6]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [7]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [8]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Distilbert Base Cased-QQP

In [9]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-cased-QQP")
tokenizer = AutoTokenizer("textattack/distilbert-base-cased-QQP")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:15<00:00, 32.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpl4tbs52f.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


# Framing fasttext attack

In [10]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [13]:
def attack_stats(attack, dataset, num_of_examples=25):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

### FastText, GreedySearch, RepeatModification, StopwordModification

In [20]:

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("glue", "qqp", "validation") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
textattack: Loading datasets dataset glue, subset qqp, split validation.


In [17]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:13,  1.85it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:07<01:28,  3.84s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:08<00:59,  2.71s/it]

...


[Succeeded / Failed / Total] 15 / 8 / 24:  96%|█████████▌| 24/25 [00:23<00:00,  1.02it/s]


[Succeeded / Failed / Total] 15 / 9 / 25: 100%|██████████| 25/25 [00:26<00:00,  1.05s/it]


[Succeeded / Failed / Total] 15 / 9 / 25: : 26it [00:26,  1.03s/it]                      
[Succeeded / Failed / Total] 16 / 9 / 26: : 26it [00:26,  1.03s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 34.62% |
| Attack success rate:          | 64.0%  |
| Average perturbed word %:     | 14.94% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 72.0   |
+-------------------------------+--------+


textattack: Attack time: 26.786608934402466s


# Framing word2vec attack

In [18]:
class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

### Word2vec, GreedySearch, RepeatModification, StopwordModification

In [21]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [22]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [23]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:01, 17.64it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:02<00:32,  1.43s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:03<00:22,  1.01s/it]

...


[Succeeded / Failed / Total] 8 / 15 / 24:  96%|█████████▌| 24/25 [00:11<00:00,  2.09it/s]


[Succeeded / Failed / Total] 8 / 16 / 25: 100%|██████████| 25/25 [00:11<00:00,  2.09it/s]


[Succeeded / Failed / Total] 8 / 16 / 25: : 26it [00:12,  2.13it/s]                      
[Succeeded / Failed / Total] 8 / 17 / 26: : 26it [00:12,  2.13it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 65.38% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 7.11%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 38.84  |
+-------------------------------+--------+


textattack: Attack time: 12.236904859542847s


### Framing Glove attack

In [24]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

### Glove, GreedySearch, RepeatModification, StopwordModification

In [25]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [26]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [27]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:04,  4.98it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:07<01:20,  3.52s/it]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:07<00:53,  2.41s/it]

...


[Succeeded / Failed / Total] 10 / 13 / 24:  96%|█████████▌| 24/25 [00:26<00:01,  1.09s/it]


[Succeeded / Failed / Total] 10 / 14 / 25: 100%|██████████| 25/25 [00:27<00:00,  1.09s/it]


[Succeeded / Failed / Total] 10 / 14 / 25: : 26it [00:27,  1.06s/it]                      
[Succeeded / Failed / Total] 11 / 14 / 26: : 26it [00:27,  1.06s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 53.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 9.12%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 37.36  |
+-------------------------------+--------+


textattack: Attack time: 27.587904930114746s


### FastText, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [28]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [29]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [30]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:04,  5.48it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:01<00:11,  1.96it/s]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:09,  2.29it/s]

...


[Succeeded / Failed / Total] 12 / 11 / 24:  96%|█████████▌| 24/25 [00:06<00:00,  3.44it/s]


[Succeeded / Failed / Total] 12 / 12 / 25: 100%|██████████| 25/25 [00:07<00:00,  3.35it/s]


[Succeeded / Failed / Total] 12 / 12 / 25: : 26it [00:07,  3.37it/s]                      
[Succeeded / Failed / Total] 12 / 13 / 26: : 26it [00:07,  3.36it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 48.0%  |
| Average perturbed word %:     | 13.96% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 33.2   |
+-------------------------------+--------+


textattack: Attack time: 7.749867677688599s


### Word2Vec, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [31]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [32]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [33]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:02,  8.26it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:07,  2.96it/s]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:05,  3.70it/s]

...


[Succeeded / Failed / Total] 8 / 15 / 24:  96%|█████████▌| 24/25 [00:05<00:00,  4.36it/s]


[Succeeded / Failed / Total] 8 / 16 / 25: 100%|██████████| 25/25 [00:05<00:00,  4.33it/s]


[Succeeded / Failed / Total] 8 / 16 / 25: : 26it [00:05,  4.35it/s]                      
[Succeeded / Failed / Total] 8 / 17 / 26: : 26it [00:05,  4.34it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 65.38% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 11.61% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 30.4   |
+-------------------------------+--------+


textattack: Attack time: 6.0062010288238525s


### Glove, GreedyWordSwapWIR RepeatModification, StopwordModification

In [34]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [35]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [36]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:04,  5.39it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:01<00:12,  1.77it/s]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:09,  2.33it/s]

...


[Succeeded / Failed / Total] 7 / 16 / 24:  96%|█████████▌| 24/25 [00:07<00:00,  3.11it/s]


[Succeeded / Failed / Total] 7 / 17 / 25: 100%|██████████| 25/25 [00:08<00:00,  3.07it/s]


[Succeeded / Failed / Total] 7 / 17 / 25: : 26it [00:08,  3.08it/s]                      
[Succeeded / Failed / Total] 8 / 17 / 26: : 26it [00:08,  3.07it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 65.38% |
| Attack success rate:          | 32.0%  |
| Average perturbed word %:     | 6.12%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 30.12  |
+-------------------------------+--------+


textattack: Attack time: 8.466029405593872s


### FastText, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [37]:
from textattack.search_methods import ParticleSwarmOptimization


# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [38]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [39]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:27,  1.13s/it]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:19<03:40,  9.59s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:20<02:27,  6.70s/it]

...


[Succeeded / Failed / Total] 14 / 9 / 24:  96%|█████████▌| 24/25 [01:02<00:02,  2.60s/it]


[Succeeded / Failed / Total] 14 / 10 / 25: 100%|██████████| 25/25 [01:12<00:00,  2.90s/it]


[Succeeded / Failed / Total] 14 / 10 / 25: : 26it [01:14,  2.85s/it]                      
[Succeeded / Failed / Total] 15 / 10 / 26: : 26it [01:14,  2.85s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 38.46% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 14.1%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 300.16 |
+-------------------------------+--------+


textattack: Attack time: 74.10370635986328s


### Word2Vec, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [40]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [41]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [42]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:01, 15.72it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:05<00:59,  2.60s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:05<00:41,  1.88s/it]

...


[Succeeded / Failed / Total] 9 / 14 / 24:  96%|█████████▌| 24/25 [00:27<00:01,  1.16s/it]


[Succeeded / Failed / Total] 9 / 15 / 25: 100%|██████████| 25/25 [00:28<00:00,  1.15s/it]


[Succeeded / Failed / Total] 9 / 15 / 25: : 26it [00:29,  1.13s/it]                      
[Succeeded / Failed / Total] 9 / 16 / 26: : 26it [00:29,  1.13s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 61.54% |
| Attack success rate:          | 36.0%  |
| Average perturbed word %:     | 8.27%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 226.96 |
+-------------------------------+--------+


textattack: Attack time: 29.320940732955933s


### Glove, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [43]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [44]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [45]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:13,  1.80it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:15<02:59,  7.80s/it]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:15<01:55,  5.27s/it]

...


[Succeeded / Failed / Total] 11 / 12 / 24:  96%|█████████▌| 24/25 [01:09<00:02,  2.88s/it]


[Succeeded / Failed / Total] 11 / 13 / 25: 100%|██████████| 25/25 [01:13<00:00,  2.92s/it]


[Succeeded / Failed / Total] 11 / 13 / 25: : 26it [01:13,  2.83s/it]                      
[Succeeded / Failed / Total] 12 / 13 / 26: : 26it [01:13,  2.83s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 48.0%  |
| Average perturbed word %:     | 10.67% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 212.48 |
+-------------------------------+--------+


textattack: Attack time: 73.5627110004425s


### FastText, GreedyWordSwapWIR, PartsofSpeech

In [46]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [47]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [48]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.54it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:01<00:16,  1.43it/s]


[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:01<00:12,  1.71it/s]

...


[Succeeded / Failed / Total] 12 / 11 / 24:  96%|█████████▌| 24/25 [00:08<00:00,  2.73it/s]


[Succeeded / Failed / Total] 12 / 12 / 25: 100%|██████████| 25/25 [00:09<00:00,  2.66it/s]


[Succeeded / Failed / Total] 12 / 12 / 25: : 26it [00:09,  2.67it/s]                      
[Succeeded / Failed / Total] 13 / 12 / 26: : 26it [00:09,  2.67it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 46.15% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 18.56% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 34.16  |
+-------------------------------+--------+


textattack: Attack time: 9.754418134689331s


### Word2Vec, GreedyWordSwapWIR, PartsofSpeech

In [49]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [50]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [51]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:04,  5.53it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:10,  2.24it/s]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:07,  2.87it/s]

...


[Succeeded / Failed / Total] 9 / 14 / 24:  96%|█████████▌| 24/25 [00:07<00:00,  3.26it/s]


[Succeeded / Failed / Total] 9 / 15 / 25: 100%|██████████| 25/25 [00:07<00:00,  3.23it/s]


[Succeeded / Failed / Total] 9 / 15 / 25: : 26it [00:08,  3.21it/s]                      
[Succeeded / Failed / Total] 9 / 16 / 26: : 26it [00:08,  3.21it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 61.54% |
| Attack success rate:          | 36.0%  |
| Average perturbed word %:     | 12.49% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 32.16  |
+-------------------------------+--------+


textattack: Attack time: 8.110139846801758s


### Glove, GreedyWordSwapWIR, PartsofSpeech

In [52]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [53]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [54]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:05,  4.18it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:01<00:17,  1.28it/s]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:12,  1.71it/s]

...


[Succeeded / Failed / Total] 10 / 13 / 24:  96%|█████████▌| 24/25 [00:12<00:00,  1.98it/s]


[Succeeded / Failed / Total] 10 / 14 / 25: 100%|██████████| 25/25 [00:12<00:00,  1.95it/s]


[Succeeded / Failed / Total] 10 / 14 / 25: : 26it [00:13,  1.93it/s]                      
[Succeeded / Failed / Total] 11 / 14 / 26: : 26it [00:13,  1.93it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 53.85% |
| Attack success rate:          | 44.0%  |
| Average perturbed word %:     | 11.14% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 30.16  |
+-------------------------------+--------+


textattack: Attack time: 13.488104343414307s


### FastText, ParticleSwarmOptimization, PartsofSpeech

In [55]:
from textattack.constraints.grammaticality import PartOfSpeech
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [56]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [57]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:13,  1.84it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:23<04:27, 11.65s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:24<03:01,  8.23s/it]

...


[Succeeded / Failed / Total] 14 / 9 / 24:  96%|█████████▌| 24/25 [02:01<00:05,  5.06s/it]


[Succeeded / Failed / Total] 14 / 10 / 25: 100%|██████████| 25/25 [02:15<00:00,  5.41s/it]


[Succeeded / Failed / Total] 14 / 10 / 25: : 26it [02:17,  5.30s/it]                      
[Succeeded / Failed / Total] 15 / 10 / 26: : 26it [02:17,  5.30s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 38.46% |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 9.68%  |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 406.2  |
+-------------------------------+--------+


textattack: Attack time: 137.79554057121277s


### Word2Vec, ParticleSwarmOptimization, PartsofSpeech

In [58]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [59]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [60]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:02,  8.13it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:14<02:51,  7.46s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:16<01:59,  5.41s/it]

...


[Succeeded / Failed / Total] 13 / 10 / 24:  96%|█████████▌| 24/25 [01:31<00:03,  3.81s/it]


[Succeeded / Failed / Total] 13 / 11 / 25: 100%|██████████| 25/25 [01:36<00:00,  3.86s/it]


[Succeeded / Failed / Total] 13 / 11 / 25: : 26it [01:42,  3.93s/it]                      
[Succeeded / Failed / Total] 13 / 12 / 26: : 26it [01:42,  3.93s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 46.15% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 13.51% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 364.56 |
+-------------------------------+--------+


textattack: Attack time: 102.11563754081726s


### Glove, ParticleSwarmOptimization, PartsofSpeech

In [61]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [62]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [63]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:15,  1.52it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:26<05:04, 13.24s/it]


[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:26<03:17,  8.97s/it]

...


[Succeeded / Failed / Total] 13 / 10 / 24:  96%|█████████▌| 24/25 [02:57<00:07,  7.41s/it]


[Succeeded / Failed / Total] 13 / 11 / 25: 100%|██████████| 25/25 [03:08<00:00,  7.54s/it]


[Succeeded / Failed / Total] 13 / 11 / 25: : 26it [03:11,  7.35s/it]                      
[Succeeded / Failed / Total] 14 / 11 / 26: : 26it [03:11,  7.35s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 42.31% |
| Attack success rate:          | 56.0%  |
| Average perturbed word %:     | 10.84% |
| Average num. words per input: | 22.5   |
| Avg num queries:              | 251.64 |
+-------------------------------+--------+


textattack: Attack time: 191.17765712738037s
